In [ ]:
def get_google_results(navegador, max_candidates):
    # Determina o número de páginas a serem processadas
    max_candidates = round(max_candidates / 5)
    perfis = []  # Lista para armazenar todos os perfis

    for i in range(max_candidates):
        # Busca elementos na página atual
        google_results = navegador.driver.find_elements(By.XPATH, '//span[@jscontroller="msmzHf"]')
        print(f"Processando candidato {i+1} de {max_candidates}")

        for result in google_results:
            
            try:
                # Tentando obter o link
                try:
                    link = result.find_element(By.TAG_NAME, "a").get_attribute("href")

                    # Normaliza o prefixo para remover qualquer idioma ou região desnecessários
                    for prefix in ["/pt", "/en", "/es", "/fr", "/de"]:  # Adicione outros idiomas, se necessário
                        link = link.replace(prefix, "")

                    # Remove o prefixo "https://br." para uniformizar os links
                    link = link.replace("https://br.", "https://")

                    # Remove qualquer prefixo antes de linkedin.com (cm., ke., etc.)
                    link = link.split('linkedin.com', 1)[-1]  # Mantém a parte após 'linkedin.com'
                    link = "https://linkedin.com" + link  # Adiciona o prefixo padrão 'https://linkedin.com'

                except:
                    link = "Link não encontrado"


                # Adicionando os dados à lista de perfis
                perfis.append({
                    "link": link,
                    "nome": "",
                    "skills": "",
                    "sobre": "",
                    "cargo": "",
                    "experiencia": [],
                    "educacao": [],
                    "certificacoes": [],
                    "contato": {
                        "email": "",
                        "telefone": "",
                        "linkedin": "",
                        "github": ""
                    }
                })
            except Exception as e:
                print(f"Erro ao processar candidato: {e}")
        
        # Avança para a próxima página
        navegador.click("ID", "pnnext")

    return perfis


In [ ]:
def update_linkedin_profile(linkedin_profile, update_profile=False):

    import dotenv
    import sqlite3
    dotenv.load_dotenv()

    cookie = {
        "name": "li_at",
        "value": os.environ["LINKEDIN_VISITOR_ID"],
        "domain": ".linkedin.com"
    }

    navegador = Navegador()

    navegador.get('https://www.linkedin.com/feed/')

    navegador.driver.add_cookie(cookie) 

    linkedin_prof = linkedin_profile

    for profile in linkedin_prof:

        #verifica se o perfil ja existe no banco de dados
        conn = sqlite3.connect("profiles.db")
        cursor = conn.cursor()
        cursor.execute("SELECT link FROM profile WHERE link = ?", (profile['link'],))
        if cursor.fetchone() is not None and update_profile == False:
            continue

        if profile['link'] == "Link não encontrado":
            continue
        print(f"Coletando dados do perfil numero {linkedin_prof.index(profile)} do total de {len(linkedin_prof)}")
        profile_url = profile['link']

        #carregar o perfil
        navegador.get(profile_url)
        
        if not is_page_exists(navegador):
            #pula o perfil
            continue

        if is_page_is_not_found(navegador):
            #pula o perfil
            continue

        soup = BeautifulSoup(navegador.driver.page_source, 'html.parser')
        #scrola a pagina e aguarda a pagina carregar
        navegador.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            # Aumentando o tempo de espera para 20 segundos
            name_element = WebDriverWait(navegador.driver, 20).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.inline.t-24.v-align-middle.break-words'))
            )
            name = name_element.text.strip() if name_element else "N/A"
        except TimeoutException:
            print("Nome não encontrado. Pode ser uma página de erro.")
            name = "N/A"

        # Verifique se name_element é um objeto BeautifulSoup e use get_text apenas se não for None
        

        try:

            photo_element = soup.select_one('img.pv-top-card-profile-picture__image--show.evi-image')
            photo_url = photo_element.get('src') if photo_element else "N/A"

            headline = soup.find('div', {'class': 'text-body-medium break-words'})
            headline = headline.get_text().strip()

            about = soup.find('div', {'class': 'display-flex ph5 pv3'}) if soup.find('div', {'class': 'display-flex ph5 pv3'}) else "N/A"
        except TimeoutException:
            headline = "N/A"
            about = "N/A"
            photo_url = "N/A"
        if about == "N/A":
            about = "N/A"
        else:
            about = about.get_text().strip()

        #atualiza o perfil
        profile['nome'] = name
        profile['skills'] = headline
        profile['sobre'] = about
        profile['foto'] = photo_url
        #coleta de experiencia
        print('-'*100)
        print(f"coletando experiencia, educacao e certificacoes de {name}")
        experiences = get_experiences(navegador, profile_url)
        profile['experiencia'] = experiences
        #coleta de educacao
        education = get_education(navegador, profile_url)
        profile['educacao'] = education
        #coleta de certificacoes
        certifications = get_certifications(navegador, profile_url)
        profile['certificacoes'] = certifications
        
        print(f"dados de {name} coletados com sucesso")
        print('-'*100)

        salvar_perfil_unico(profile)

    navegador.close()
    
    return linkedin_prof


In [1]:
import requests

response = requests.post(
    "http://127.0.0.1:3000//searchprofiles",  # Substitua pelo endpoint correto
    json={
        "cargos": ["Desenvolvedor Backend"],
        "habilidades": ["Django", "FastAPI"],
        "ferramentas": ["Docker", "Kubernetes"],
        "localizacoes": ["Brasil"],
        "max_interactions": 5,
        "job_bubble_id": "1738117388263x970142495908560900",
    },
    headers={"Content-Type": "application/json"}
)

print(response.json())


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=3000): Max retries exceeded with url: /searchprofiles (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b02053f4e90>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [2]:
import requests

BASE_URL = "http://127.0.0.1:3000"  # Altere para o IP correto se necessário

# Dados a serem enviados
json_data = {
    "job_id": "aaaaaa",
    "max_candidates": 5,
    "cargos": ["Desenvolvedor Front End"],
    "habilidades": ["Django", "FastAPI"],
    "ferramentas": ["Docker", "Kubernetes"],
    "localizacoes": ["Brasil", "Portugal"],
    "max_interactions": 5,
    "job_bubble_id": "1738117388263x970142495908560900"
}

# Fazendo a requisição POST
response = requests.post(f"{BASE_URL}/add_job", json=json_data)

# Exibindo a resposta
print("Status Code:", response.status_code)
print("Response JSON:", response.json())


Status Code: 201
Response JSON: {'inserted_id': 1, 'job_id': 'aaaaaa', 'message': 'Job adicionado'}


In [45]:
import requests

BASE_URL = "http://127.0.0.1:3000"  # Altere para o IP correto se necessário

# Dados a serem enviados
data = {
    "job_id": "job_123",
    "status": 1
}

update_job_status("job_123", 0)

Job job_123 atualizado para status 0


In [111]:
import requests

BASE_URL = "http://127.0.0.1:3000"  # Altere para o IP correto se necessário

# Dados a serem enviados
json_data = {
    "candidate_cv_info": "sem dados",
    "job_info": "sem dados"
}

# Fazendo a requisição POST
response = requests.post(f"{BASE_URL}/avaliate_candidate", json=json_data)

# Exibindo a resposta
print("Status Code:", response.status_code)
print(response.json())


Status Code: 200
{'alignment': {'hard_skills': 'Não atendido', 'soft_skills': 'Não atendido'}, 'justification': 'Não foi possível avaliar a aderência do candidato à vaga devido à ausência total de informações sobre o candidato e os requisitos da vaga.', 'missing_information': 'Informações sobre o candidato, incluindo experiências profissionais, habilidades, formações e outras qualificações; detalhes da vaga, incluindo requisitos obrigatórios, desejáveis, responsabilidades e competências esperadas.', 'recommendation': 'Não recomendável', 'score': 0, 'strengths': [], 'weaknesses': []}


In [98]:
from crewai import Agent, Crew, Process, Task, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool, FileReadTool, PDFSearchTool
from dotenv import load_dotenv
import os
import contextlib
import warnings
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests

load_dotenv()

def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()
    
def parse_json_from_crew_output(raw_text):
    # Remove espaços em branco no início/fim
    cleaned = raw_text.strip()
    
    # Remove o bloco de código markdown se existir
    if cleaned.startswith("```json"):
        cleaned = cleaned[len("```json"):].strip()
    if cleaned.endswith("```"):
        cleaned = cleaned[:-3].strip()
    
    # Agora converte a string limpa para JSON
    return json.loads(cleaned)


def avaliation_candidate(candidate_cv_info, job_info):

    # Carregando os dados dos arquivos
    candidate_cv_info = load_file_content('candidate_info.txt')
    job_info = load_file_content('job_info.txt')

    API_KEY = os.getenv("OP_AI_KEY")
    os.environ["OPENAI_API_KEY"] = "NA"

    gpt = LLM(model="gpt-4o-mini", temperature=0.5, api_key=API_KEY)

    ollama = LLM(
        model="ollama/llama3.1",  # Especificando o provedor
        base_url="http://localhost:11434",
        temperature=0.5
    )
    GROQ_API_KEY="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"

    groq = LLM(
        model="groq/llama-3.1-70b-versatile",
        api_key=GROQ_API_KEY,
        temperature=0.5
    )

    def create_session():
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        return session

    selected_llm = gpt

    json_schema = """
    {
    "score": "Pontuação de 0 a 100, indicando a aderência do candidato aos requisitos da vaga",
    "strengths": "Lista detalhada de pontos fortes do candidato em relação aos requisitos da vaga",
    "weaknesses": "Lista de lacunas ou áreas onde o candidato não atende aos requisitos da vaga",
    "alignment": {
        "hard_skills": "Nível de alinhamento das hard skills do candidato com os requisitos obrigatórios e desejáveis",
        "soft_skills": "Nível de alinhamento das soft skills do candidato com os requisitos comportamentais da vaga"
    },
    "missing_information": "Quaisquer informações ausentes em {candidate_cv_info} ou {job_info} que dificultaram a avaliação",
    "justification": "Resumo textual que explica o score atribuído com base nos requisitos da vaga e qualificações do candidato",
    "recommendation": "Recomendação final, categorizada como 'Altamente recomendável', 'Recomendável com reservas' ou 'Não recomendável'"
    }

    """

    Recrutador = Agent(
        role="Recrutador",
        goal="Avaliar o currículo do candidato e o perfil do cargo para determinar se o candidato é adequado para o cargo",
        backstory="Você é um recrutador experiente e sempre busca avaliar o currículo do candidato e o perfil do cargo para determinar se o candidato é adequado para o cargo",
        verbose=True,
        allow_delegation=False,
        llm=selected_llm
    )

    Avaliar_candidato = Task(
        description=f"""Aja como um especialista em Recursos Humanos e recrutamento com experiência em avaliação de candidatos para diferentes tipos de vagas. Sua tarefa é realizar uma análise objetiva e detalhada, comparando as qualificações e habilidades do candidato com os requisitos da vaga, considerando os seguintes aspectos:

    Hard skills: Experiências profissionais, formações acadêmicas, certificações e competências técnicas exigidas pela vaga.
    Soft skills: Competências comportamentais e interpessoais relevantes, como comunicação, trabalho em equipe, resolução de problemas e liderança.
    Aderência geral: Compatibilidade com os requisitos obrigatórios e desejáveis descritos na vaga.
    Dados Fornecidos:
    {candidate_cv_info}: Informações sobre o candidato, incluindo experiências profissionais, habilidades, formações e outras qualificações.
    {job_info}: Detalhes da vaga, incluindo requisitos obrigatórios, desejáveis, responsabilidades e competências esperadas.
    Suas Tarefas:
    Comparar: Avalie as informações do candidato com os requisitos obrigatórios e desejáveis da vaga, identificando lacunas e alinhamentos.
    Classificar: Atribua um score de 0 a 100, indicando o grau de aderência do candidato à vaga.
    Priorizar: Dê preferência a candidatos que atendam aos requisitos obrigatórios e demonstrem diferenciais significativos.
    Relatar Dados Faltantes: Se informações cruciais estiverem ausentes em {candidate_cv_info} ou {job_info}, mencione isso no output.
    Formato da Resposta:
    Sua avaliação deve ser concisa e apresentada no seguinte formato JSON:

    {json_schema}


    Orientações Adicionais:
    Para cada hard skill, avalie seu nível de aderência como 'Totalmente atendido', 'Parcialmente atendido' ou 'Não atendido'.
    Para cada soft skill, avalie seu impacto potencial na execução das responsabilidades da vaga.
    Se o candidato apresentar diferenciais que superem algumas lacunas, destaque isso no campo strengths.
    Caso os dados da vaga ou do candidato estejam mal estruturados, identifique as inconsistências e forneça uma análise baseada nas informações disponíveis.
    """,
        expected_output="JSON",
        agent=Recrutador,
        llm=selected_llm,
    )


    crew = Crew(
        name="Avaliar candidato",
        description="Avaliar o candidato para o cargo",
        tasks=[Avaliar_candidato],
        agents=[Recrutador],
        llm=selected_llm,
        verbose=True,
         expected_output="JSON",
    )

    crew_output = crew.kickoff()

    rew_json = parse_json_from_crew_output(crew_output.raw)

    return rew_json

response = avaliation_candidate("a", "b")



# Agent: Recrutador
## Task: Aja como um especialista em Recursos Humanos e recrutamento com experiência em avaliação de candidatos para diferentes tipos de vagas. Sua tarefa é realizar uma análise objetiva e detalhada, comparando as qualificações e habilidades do candidato com os requisitos da vaga, considerando os seguintes aspectos:

    Hard skills: Experiências profissionais, formações acadêmicas, certificações e competências técnicas exigidas pela vaga.
    Soft skills: Competências comportamentais e interpessoais relevantes, como comunicação, trabalho em equipe, resolução de problemas e liderança.
    Aderência geral: Compatibilidade com os requisitos obrigatórios e desejáveis descritos na vaga.
    Dados Fornecidos:
    {
    "curriculos": {
        "curriculos": [
            {
                "about": {
                    "contact": {
                        "address": [
                            {
                                "city": "Rio de Janeiro",
                 

In [101]:
response


{'score': 45,
 'strengths': ['Experiência em HTML5, CSS3 e JavaScript, que são essenciais para desenvolvimento front-end.',
  'Certificação em Design UX/UI, que pode contribuir para a criação de interfaces amigáveis.',
  'Habilidades em comunicação e trabalho em equipe, adquiridas em experiência anterior.',
  'Proficiente em inglês (nível C1), o que pode facilitar a comunicação em ambientes internacionais.'],
 'weaknesses': ['Falta de experiência específica com Next.js, Vue.js e Tailwind CSS.',
  'Não possui experiência com HubSpot CMS, HubL, HubDb e funções serverless.',
  'Experiência profissional total inferior a 5 anos, sendo apenas 3 anos de experiência prática.',
  'Não apresenta conhecimento em integração de APIs REST/GraphQL.'],
 'alignment': {'hard_skills': 'Parcialmente atendido',
  'soft_skills': 'Totalmente atendido'},
 'missing_information': 'Não foram fornecidos detalhes sobre projetos anteriores que utilizem as tecnologias exigidas pela vaga.',
 'justification': 'O candi